In [1]:
import os
os.environ['USE_PYGEOS'] = '0'

import pandas as pd
import geopandas as gpd
import rasterio
import fiona

from pyproj import CRS, Transformer
from shapely.geometry import Point, LineString, Polygon
from shapely.wkb import loads, dumps
from rasterio.mask import mask
from rasterio.plot import show

In [2]:
# Define AOI and CRS
aoi  = 'pidie'
utm_epsg = 32647

In [3]:
# Load rasters 
lco = r'G:/My Drive/Spatial data/LC/COPERNICUS_100M_PIDIE.tif'
pop = r'G:/My Drive/Spatial data/POP/WORLDPOP_100M_PIDIE.tif'

# Load shapefiles
adm = gpd.read_file(r'G:/My Drive/Spatial data/SHP Batas Desa/Pidie/admDesa.shp')
mkm = gpd.read_file(r'G:/My Drive/Spatial data/SHP RBI/Pidie/Pidie/RBI50K_PEMUKIMAN_AR.shp')
jln = gpd.read_file(r'G:/My Drive/Spatial data/SHP RBI/Pidie/Pidie/RBI50K_JALAN_LN_50K.shp')
sng = gpd.read_file(r'G:/My Drive/Spatial data/SHP RBI/Pidie/Pidie/RBI50K_SUNGAI_LN_50K.shp')

# Load tables
bps = pd.read_excel(r'G:/My Drive/Spatial data/Populasi/Populasi Pidie BPS 2016.xlsx')
fks = pd.read_excel(r'G:/My Drive/Spatial data/Puskesmas/geoloc_240421.xlsx')

In [4]:
# Import necessary libraries for coordinate reference systems and transformations
utm_proj = CRS.from_user_input(utm_epsg)  # Create a UTM projection based on user input EPSG code
transformer = Transformer.from_crs('EPSG:4326', utm_proj, always_xy=True)  # Initialize a transformer from WGS84 to UTM

# Function to convert coordinates of a LineString from WGS84 to UTM
def convert_linestring_coordinates(linestring):
    obj_coords = [transformer.transform(x, y) for x, y in linestring.coords]  # Transform each coordinate pair
    linestring = LineString(obj_coords)  # Create a new LineString with transformed coordinates
    return linestring  # Return the transformed LineString

# Function to convert coordinates of a Polygon from WGS84 to UTM
def convert_polygon_coordinates(polygon):
    obj_coords = [transformer.transform(x, y) for x, y in polygon.exterior.coords]  # Transform each coordinate pair of the polygon's exterior
    polygon = Polygon(obj_coords)  # Create a new Polygon with transformed coordinates
    return polygon  # Return the transformed Polygon

# Function to transform geometries in a GeoDataFrame to UTM
def transform_to_utm(gdf):
    gdf = gdf.explode(index_parts=False)  # Explode multi-part geometries into single parts
    gdf.geometry = loads(dumps(gdf.geometry, output_dimension=2))  # Ensure geometries are 2D
    if gdf.geom_type.nunique() == 1:  # Check if all geometries are of the same type
        if gdf.geom_type.unique()[0] == 'LineString':  # If all geometries are LineStrings
            gdf.geometry = gdf.geometry.apply(convert_linestring_coordinates)  # Convert coordinates
        elif gdf.geom_type.unique()[0] == 'Polygon':  # If all geometries are Polygons
            gdf.geometry = gdf.geometry.apply(convert_polygon_coordinates)  # Convert coordinates
        else:
            raise ValueError('gdf.geom_type is not LineString or Polygon')  # Raise error if neither
    else:
        raise ValueError('gdf.geom_type is not unique')  # Raise error if geometries are mixed types
    gdf.set_crs(utm_proj, allow_override=True, inplace=True)  # Set CRS to UTM
    return gdf  # Return transformed GeoDataFrame

# Function to get the Area of Interest (AOI) from a GeoDataFrame
def get_aoi(gdf):
    gdf['NAMOBJ'] = gdf.NAMOBJ.apply(lambda x: x.split('/')[0]) # Update 'NAMOBJ' to contain only the substring before the first '/'
    gdf['WADMKC'] = gdf.WADMKC.fillna(gdf.NAMOBJ).apply(lambda x: x.split('/')[0]) # Fill NaNs in 'WADMKC', then update to substring before the first '/'
    gdf = transform_to_utm(gdf)  # Transform geometries to UTM
    aoi = gdf.dissolve().explode(index_parts=False)  # Dissolve geometries into a single geometry and explode multi-parts
    aoi['area'] = aoi.geometry.area  # Calculate area of each geometry
    aoi = aoi[aoi.area == aoi.area.max()][['WADMKK', 'geometry']]  # Select the largest area geometry
    gdf = gdf.sjoin(aoi[['geometry']])[['NAMOBJ', 'WADMKC', 'WADMKK', 'geometry']].reset_index(drop=True)  # Spatial join with AOI
    return aoi, gdf  # Return AOI and updated GeoDataFrame

# Function to clip a GeoDataFrame with a LineString AOI
def line_cookie_cutter(aoi, gdf, columns_list):
    gdf = transform_to_utm(gdf)  # Transform geometries to UTM
    gdf = gdf.clip(aoi, keep_geom_type=True)[columns_list].sort_index().explode(index_parts=False).reset_index(drop=True)  # Clip geometries with AOI and retain specified columns
    return gdf  # Return clipped GeoDataFrame

# Function to subtract an AOI from a GeoDataFrame of polygons
def poly_cookie_cutter(aoi, gdf):
    gdf = transform_to_utm(gdf)  # Transform geometries to UTM
    gdf = gpd.overlay(aoi, gdf, how='difference').explode(index_parts=False)  # Perform difference overlay with AOI
    return gdf  # Return resulting GeoDataFrame

# Function to convert a DataFrame to a GeoDataFrame
def to_gdf(df):
    gdf = df.copy()  # Create a copy of the DataFrame
    if gdf.LONGITUDE.dtype == 'object':  # If LONGITUDE column is of type object
        gdf.LONGITUDE = gdf.LONGITUDE.str.replace('‐', '-').str.replace(',', '.').astype(float)  # Replace special characters and convert to float
    if gdf.LATITUDE.dtype == 'object':  # If LATITUDE column is of type object
        gdf.LATITUDE = gdf.LATITUDE.str.replace('‐', '-').str.replace(',', '.').astype(float)  # Replace special characters and convert to float
    else:
        pass  # If LATITUDE is already float, do nothing
    obj_coords = [Point(transformer.transform(*xy)) for xy in zip(gdf.LONGITUDE, gdf.LATITUDE)]  # Transform coordinates to UTM
    gdf = gpd.GeoDataFrame(gdf, geometry=obj_coords, crs=utm_proj)  # Create a GeoDataFrame with transformed coordinates
    return gdf  # Return the GeoDataFrame

# Function to crop a raster using a shapefile and save the result
def raster_cookie_cutter(cutter_shp_dir, in_raster_dir, out_raster_dir):    
    # Open the shapefile containing the shapes to use for cutting the raster
    with fiona.open(cutter_shp_dir, 'r') as shapefile:
        shapes = [feature['geometry'] for feature in shapefile]  # Extract geometries from the shapefile

    # Open the input raster file
    with rasterio.open(in_raster_dir) as src:
        # Use the geometries to mask and crop the raster
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
        out_meta = src.meta  # Get metadata from the input raster

    # Update the metadata with new dimensions and transformation
    out_meta.update({
        'driver': 'GTiff',  # Set the output driver to GeoTIFF
        'height': out_image.shape[1],  # Update height
        'width': out_image.shape[2],  # Update width
        'transform': out_transform  # Update transformation matrix
    })

    # Write the cropped image to a new raster file
    with rasterio.open(out_raster_dir, 'w', **out_meta) as dest:
        dest.write(out_image)  # Save the masked and cropped image

    # Print success message and return it
    return(print('Success!'))

In [5]:
aoi_bounds, adm_bounds = get_aoi(adm) # Batas administrasi kabupaten
akc_bounds = adm_bounds.dissolve(['WADMKC', 'WADMKK'], as_index=False).drop(columns='NAMOBJ') # Batas administrasi kecamatan
akc_bounds = akc_bounds.merge(bps, on='WADMKC').reset_index(names='IDOBJ') # Menambahkan kolom populasi ke shapefile
ads_bounds = adm_bounds.reset_index(names='IDOBJ') # Batas administrasi desa
mkm_bounds = poly_cookie_cutter(aoi_bounds, mkm) # Wilayah non-permukiman
jln_aoi = line_cookie_cutter(aoi_bounds, jln, ['REMARK', 'geometry']) # Mengambil objek jalan dalam wilayah aoi
jln_class = {'Jalan Lain':5000, 'Jalan Setapak':4000, 'Jalan Lokal':3000, 'Jalan Kolektor':2000,  'Jalan Arteri':1000} # Kode kelas jalan
jln_aoi.insert(0, 'KELASJLN', jln_aoi.REMARK.map(jln_class)) # Menambahkan kode kelas jalan ke shapefile
sng_filtered = sng[(~sng.NAMOBJ.isnull())|(sng.REMARK=='Sungai')] # Memilih objek sungai berdasarkan nama dan kelas
sng_aoi = line_cookie_cutter(aoi_bounds, sng_filtered, ['REMARK', 'geometry']) # # Mengambil objek sungai dalam wilayah aoi
fks_aoi = to_gdf(fks[(fks.kabupaten=='Pidie')&(~fks.LATITUDE.isnull())]) # Fasilitas kesehatan di aoi

# aoi_bounds.to_file(aoi_dir)
# akc_bounds.to_file(akc_dir)
# ads_bounds.to_file(ads_dir)
# mkm_bounds.to_file(mkm_dir)
# jln_aoi.to_file(jln_dir)
# sng_aoi.to_file(sng_dir)
# fks_aoi.to_file(fks_dir)

In [6]:
aoi_dir = r'G:/My Drive/Spatial data/Pidie/batas_wilayah/batas_wilayah_pidie.shp'
lco_dir = r'G:/My Drive/Spatial data/Pidie/raster/tutupan_lahan_pidie.tif'
pop_dir = r'G:/My Drive/Spatial data/Pidie/raster/sebaran_penduduk_pidie.tif'

raster_cookie_cutter(aoi_dir, lco, lco_dir) # Raster tutupan lahan dalam aoi 
raster_cookie_cutter(aoi_dir, pop, pop_dir) # Raster populasi dalam aoi 

Success!
Success!
